In [1]:
# Install required packages
!pip install folium
!pip install geopandas
import folium
from folium import GeoJson
import pandas as pd
import geopandas as gpd
from IPython.display import display
import json

# Data desa dan jumlah KK Kecamatan Gunung Sindur
data = {
    'Desa': [
        'Jampang', 'Cibadung', 'Cibinong', 'Cidokom', 'Pedurenan',
        'Gunung Sindur', 'Curug', 'Rawakalong', 'Pengasinan', 'Pabuaran'
    ],
    'KK': [1460, 2077, 2977, 2044, 1867, 2594, 3226, 3689, 2492, 2339]
}

# Buat DataFrame
df = pd.DataFrame(data)

# Peta dasar berpusat di Gunung Sindur
m = folium.Map(
    location=[-6.35, 106.68],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Warna untuk setiap desa berdasarkan jumlah KK
colors = {
    'Jampang': '#FFF7BC',      # Kuning sangat muda
    'Cibadung': '#FEE391',     # Kuning muda
    'Cibinong': '#FEC44F',     # Kuning
    'Cidokom': '#FE9929',      # Oranye muda
    'Pedurenan': '#EC7014',    # Oranye
    'Gunung Sindur': '#CC4C02', # Oranye tua
    'Curug': '#993404',        # Merah oranye
    'Rawakalong': '#7F2704',   # Merah tua
    'Pengasinan': '#662506',   # Merah sangat tua
    'Pabuaran': '#993404'      # Merah oranye
}

# Koordinat Desa
desa_coordinates = {
    'Jampang': [
        [106.665, -6.335], [106.675, -6.332],
        [106.680, -6.338], [106.672, -6.342],
        [106.665, -6.340], [106.665, -6.335]
    ],
    'Cibadung': [
        [106.675, -6.332], [106.685, -6.330],
        [106.690, -6.335], [106.680, -6.338],
        [106.675, -6.332]
    ],
    'Cibinong': [
        [106.685, -6.330], [106.698, -6.332],
        [106.698, -6.338], [106.690, -6.335],
        [106.685, -6.330]
    ],
    'Cidokom': [
        [106.665, -6.340], [106.672, -6.342],
        [106.678, -6.348], [106.670, -6.352],
        [106.665, -6.348], [106.665, -6.340]
    ],
    'Pedurenan': [
        [106.672, -6.342], [106.680, -6.338],
        [106.690, -6.335], [106.698, -6.338],
        [106.698, -6.342], [106.688, -6.348],
        [106.678, -6.348], [106.672, -6.342]
    ],
    'Gunung Sindur': [
        [106.698, -6.338], [106.705, -6.340],
        [106.705, -6.348], [106.698, -6.342],
        [106.698, -6.338]
    ],
    'Curug': [
        [106.665, -6.348], [106.670, -6.352],
        [106.675, -6.358], [106.668, -6.362],
        [106.665, -6.360], [106.665, -6.348]
    ],
    'Rawakalong': [
        [106.670, -6.352], [106.678, -6.348],
        [106.688, -6.348], [106.698, -6.348],
        [106.698, -6.358], [106.675, -6.358],
        [106.670, -6.352]
    ],
    'Pengasinan': [
        [106.698, -6.348], [106.705, -6.348],
        [106.705, -6.358], [106.698, -6.358],
        [106.698, -6.348]
    ],
    'Pabuaran': [
        [106.705, -6.340], [106.712, -6.342],
        [106.712, -6.358], [106.705, -6.358],
        [106.705, -6.348], [106.705, -6.340]
    ]
}

# Buat GeoJSON FeatureCollection
geojson_data = {
    "type": "FeatureCollection",
    "features": []


# Buat feature untuk setiap desa
for desa in data['Desa']:
    kk = df[df['Desa'] == desa]['KK'].values[0]
    feature = {
        "type": "Feature",
        "properties": {
            "name": desa,
            "kk": f"{kk:,}"
        },
        "geometry": {
            "type": "Polygon",
            "coordinates": [desa_coordinates[desa]]
        }
    }
    geojson_data["features"].append(feature)

# Function untuk styling area dengan transparansi
def style_function(feature):
    desa = feature['properties']['name']
    return {
        'fillColor': colors[desa],
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.4,
        'dashArray': '5, 5'
    }

# GeoJSON layer ke peta
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['name', 'kk'],
        aliases=['Desa:', 'Jumlah KK:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    ),
    popup=folium.GeoJsonPopup(
        fields=['name', 'kk'],
        aliases=['Desa:', 'Jumlah KK:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
).add_to(m)

# Legenda ke peta
legend_html = '''
<div style="position: fixed;
            bottom: 50px; right: 50px;
            border:2px solid grey; z-index:9999; font-size:12px;
            background-color:white;
            padding: 10px;
            opacity: 0.8;
            border-radius: 6px;
            box-shadow: 0 0 15px rgba(0,0,0,0.2);
            ">
    <p style="margin: 0; padding-bottom: 5px;"><strong>Jumlah KK per Desa</strong></p>
'''

# Item legenda untuk setiap desa
for desa, kk in zip(data['Desa'], data['KK']):
    legend_html += f'''
    <div style="display: flex; align-items: center; margin: 2px;">
        <span style="background-color: {colors[desa]};
                     border: 1px solid black;
                     display: inline-block;
                     width: 12px;
                     height: 12px;
                     margin-right: 5px;
                     opacity: 0.4;">
        </span>
        <span>{desa}: {kk:,} KK</span>
    </div>
    '''
legend_html += '</div>'
m.get_root().html.add_child(folium.Element(legend_html))

# Tampilkan peta
display(m)